In [21]:
from dxfwrite import DXFEngine as dxf
import numpy as np
import matplotlib.pyplot as plt
import microfluidics_class as mf
from microfluidics_class import Design
from microfluidics_class import Feature
from copy import deepcopy

In [22]:
#initialize the design object and set a file name
design = Design()
design.file = '/Users/guillaume/Google Drive/PostDocBasel/PyMicrofluidics/3layerMoM_class.dxf'

#Each layer is defined as a dictionary that is itself stored in a dictionary with a useful name. Each layer dictionary
#has the following fields
#name: layer name stored in DXF file
#color: color assigned to layer
#inversion: 0 == nothing happens, 1 == the layer is rotated 180° on the wafer (for multilayers)
design.add_layer('layer_deep', {'name':'layer_6um','color':1, 'inversion':0})
design.add_layer('layer_growth', {'name':'layer_900nm','color':2, 'inversion':0})
design.add_layer('layer_shallow', {'name':'layer_200nm','color':3, 'inversion':0})
design.add_layer('layer_sizeref', {'name':'size_ref','color':4, 'inversion':0})

In [23]:
#where to move the complete design
origins = [np.array([-25000,0]),np.array([-25000,-20000]),np.array([-25000,20000])]
#gc_widht is a list of growth channel widths in um
widths_GC = [0.8,0.8,1.0,1.0,1.1,1.1]

In [24]:
#define position of all the features. Here the [0,0] position is at the dial-awawe
#radius of large channels
lc_rad = 25
#positions relative to origin
pos_inlet = np.array([-2100,-2600])
#left and right x limit of channel features (except OF)
leftlim = 700
rightlim = 11670

#inlet
x_inlet = -2300
y_inlet = -2600
#dial-awawe
separation = 10#separation between OF channels and channel leading to serpentine
#serpentine
x_serpentine_start = 1000
y_serpentine_start = -500
#BF outlet 
x_outlet_BF = 19500
#CF outlet 
x_outlet_CF = 15500
#OF outlet 
x_outlet_OF = 23500

#OF at dial-awawe
y_dial_OF = -1.5*lc_rad-separation
#OF main distance
y_main_OF = -3400
#BF main y distance
y_main_BF1 = -2500
y_main_BF2 = -1740
y_main_BF3 = -1150

#CF main y distance
y_main_CF1 = y_main_BF1+120
y_main_CF2 = y_main_CF1+280
y_main_CF3 = y_main_BF3+120
y_main_CF4 = y_main_CF3+280

#%part of the growth channel overlapping with main channels
overlap_GC = 3

In [25]:
#growth channel parameters
length_GC = 40;#channel length (including overlapping region)
num_GC = 320;#number of channels per size
space_GC = 5;#space between single channels
space_series_GC = 100;#space between channel series
#pos_channelseries1 = junction+[1000,-2600];%lower left position of growth channel series 1
#pos_channelseries2 = pos_channelseries1+[0,1300];%lower left position of growth channel series 2

In [26]:
#abberviations
#OF == overflow
#CF == channel flow
#BF == back flow
#GC == growth channel
#NL = number layer

In [27]:
#define the paths of all features as well as some special features like channel arrays

#inlet channels
xy_inlet = [[x_inlet, y_inlet],[-100,-lc_rad-separation],[0,-lc_rad-separation]]
curv_inlet = [0,50,0]

#dial-awawe to serpentine
xy_dial_serpentine = [[0,0],[460,0],[x_serpentine_start-100,y_serpentine_start],[x_serpentine_start,y_serpentine_start]]
curv_dial_serpentine = [0,50,50,0]

#serpentine
serpentine = Feature.serpentine(7, 150, lc_rad, 10000,75,[x_serpentine_start,y_serpentine_start],'left','bottom', 'horizontal')

#serpentine to CF
xy_serpentine_to_CF = [[x_serpentine_start+10000,y_serpentine_start+(7-1)*150],[x_serpentine_start+10000+300,y_serpentine_start+(7-1)*150],
                      [np.max(serpentine.coord[0])+300,0],[rightlim-200,0]]
curv_serpentine_to_CF = [0,50,50,0]

#overflow channel
xy_OF = [[0,y_dial_OF],[100,y_dial_OF],[300,-300],[300,y_main_OF],[x_outlet_OF,y_main_OF],[x_outlet_OF,0]]
curv_OF = [0,50,50,50,50,0]
rad_OF = 12.5

#back flow channel
xy_BF = [[x_outlet_BF,0],[x_outlet_BF,y_main_BF1],[leftlim,y_main_BF1],[leftlim,y_main_BF2],
         [rightlim-260,y_main_BF2],[rightlim-260,y_main_BF3],[leftlim,y_main_BF3]]
curv_BF = [0,50,50,50,50,50,0]
rad_BF = lc_rad

#regular flow channel
xy_CF = [[x_outlet_CF,0],[rightlim,y_main_CF1],[leftlim+100,y_main_CF1],[leftlim+100,y_main_CF2],
         [rightlim,y_main_CF2],[rightlim,y_main_CF3],[leftlim+100,y_main_CF3],[leftlim+100,y_main_CF4],
        [rightlim,y_main_CF4],[rightlim,-100],[rightlim-100,0],[rightlim-200,0]]
curv_CF = [0,50,50,50,50,50,50,50,50,50,50,0]
rad_CF = lc_rad

#global filter shape
xy_filter = [[leftlim, y_main_BF3-lc_rad],[leftlim,y_main_CF4],[leftlim+100,y_main_CF4]]
curv_filter = [0,25,0]
rad_filter = 25

#growith channels
GC_1 = Feature.channel_array(length_GC, num_GC, space_GC, space_series_GC, widths_GC, 
                         [leftlim+300,y_main_CF1-lc_rad+overlap_GC],1)
GC_2 = Feature.channel_array(length_GC, num_GC, space_GC, space_series_GC, widths_GC, 
                        [leftlim+300,y_main_CF3-lc_rad+overlap_GC],1)

#challow comb structure at the end of growth channels  
rad_comb = 0.5
comb_long_1 = Feature.define_tube([[np.min(GC_1.coord[0][:,0])-2,np.min(GC_1.coord[0][:,1])+rad_comb+2],[np.max(GC_1.coord[-1][:,0])+2,np.min(GC_1.coord[0][:,1])+rad_comb+2]],[0,0],rad_comb)
comb_long_2 = Feature.define_tube([[np.min(GC_2.coord[0][:,0])-2,np.min(GC_2.coord[0][:,1])+rad_comb+2],[np.max(GC_2.coord[-1][:,0])+2,np.min(GC_2.coord[0][:,1])+rad_comb+2]],[0,0],rad_comb)
comb_teeth_1 = Feature.channel_array(10, num_GC, space_GC, space_series_GC, [1 for i in range(len(widths_GC))], 
                         [leftlim+300+0.5*space_GC,np.min(GC_1.coord[0][:,1])+2],1)
comb_teeth_2 = Feature.channel_array(10, num_GC, space_GC, space_series_GC, [1 for i in range(len(widths_GC))], 
                         [leftlim+300+0.5*space_GC,np.min(GC_2.coord[0][:,1])+2],1)



In [8]:
#create layer with numbers and supporting pillars
miny_comb = np.min(comb_teeth_1.coord[0][:,1])
y_NL = y_main_BF1+0.5*(np.abs((miny_comb-y_main_BF1))+lc_rad)
rad_NL = (miny_comb-y_main_BF1-lc_rad)/2+3
NL_back_1 = Feature.define_tube([[np.min(comb_teeth_1.coord[0][:,0])-20,y_NL],[np.max(comb_teeth_1.coord[-1][:,0])+20,y_NL]],[0,0],rad_NL)

num_array = Feature.number_array(5, num_GC, space_GC, space_series_GC, len(widths_GC), 
                         [leftlim+300,y_NL],10, rotation = -np.pi/2)

support_len = 20
support_array = Feature.channel_array(support_len, num_GC, space_GC, space_series_GC, [2 for i in range(len(widths_GC))],
                                 [leftlim+300+0.5*space_GC,y_NL+support_len/2],2)
support_array.coord = [support_array.coord[x] for x in range(len(support_array.coord)) if np.mod(x+0,5)*np.mod(x,5)!=0] 

features_to_invert = Feature.combine_features(num_array,support_array)
num_pad = Feature.reverse_feature(features_to_invert,NL_back_1.coord[0])

#copy this number layer at each location of channel arrays (don't use mirroring as it flips numbers)
num_pad2 = num_pad.copy()
for x in range(len(num_pad2.coord)):
    num_pad2.coord[x] = np.array([y + np.array([0,y_main_BF3-y_main_BF1]) for y in num_pad2.coord[x]])
    
num_pad3 = num_pad.copy()
y_target = -y_NL + (-y_main_BF1) - lc_rad -rad_NL+3
for x in range(len(num_pad2.coord)):
    num_pad3.coord[x] = np.array([y + np.array([0,y_target]) for y in num_pad3.coord[x]])
    
num_pad4 = num_pad.copy()
y_target = -y_NL + (-y_main_BF3) - lc_rad -rad_NL+3
for x in range(len(num_pad2.coord)):
    num_pad4.coord[x] = np.array([y + np.array([0,y_target]) for y in num_pad4.coord[x]])

In [28]:
#design is a dictionnary of features to draw. Each feature is itself a dictionary entry with a freely chosen name and 
#that has the following content:
#1. 'coord': either a 2D numpy array or a list of 2D numpy arrays
#2. 'layer': a dictionnary e.g. {'name':'layer_6um','color':1, 'inversion':0}
#3. 'mirror': an optional field that gives the position of the horizontal axis around which to mirror the feature

design.add_feature('inlet_channel',Feature.define_tube(xy_inlet,curv_inlet,lc_rad).set_layer('layer_deep').set_mirror(0))
design.add_feature('over_flow',Feature.define_tube(xy_OF,curv_OF,rad_OF).set_layer('layer_deep').set_mirror(0))
design.add_feature('back_flow',Feature.define_tube(xy_BF,curv_BF,rad_BF).set_layer('layer_deep').set_mirror(0))
design.add_feature('channel_flow',Feature.define_tube(xy_CF,curv_CF,rad_CF).set_layer('layer_deep').set_mirror(0))
design.add_feature('serpentine',Feature.serpentine(7, 150, lc_rad, 10000,75,[x_serpentine_start,y_serpentine_start], 'left','bottom', 'horizontal').set_layer('layer_deep'))
design.add_feature('dial_to_serpentine',Feature.define_tube(xy_dial_serpentine,curv_dial_serpentine,lc_rad).set_layer('layer_deep'))
design.add_feature('serpentine_to_CF',Feature.define_tube(xy_serpentine_to_CF,curv_serpentine_to_CF,lc_rad).set_layer('layer_deep'))

design.add_feature('punching_entry',Feature.circular_punching(20,250,[x_inlet,y_inlet]).set_layer('layer_deep').set_mirror(0))
design.add_feature('punching_BF',Feature.circular_punching(20,250,[x_outlet_BF,0]).set_layer('layer_deep'))
design.add_feature('punching_OF',Feature.circular_punching(20,250,[x_outlet_OF,0]).set_layer('layer_deep'))
design.add_feature('punching_CF',Feature.circular_punching(20,250,[x_outlet_CF,0]).set_layer('layer_deep'))

design.add_feature('growth_channel_1',GC_1.set_layer('layer_growth').set_mirror(0))
design.add_feature('growth_channel_2',GC_2.set_layer('layer_growth').set_mirror(0))

design.add_feature('filter',Feature.patterned_region(Feature.define_tube(xy_filter,curv_filter,rad_filter).coord[0],1,7).set_layer('layer_shallow').set_mirror(0))

design.add_feature('comb_long1', comb_long_1.set_layer('layer_shallow').set_mirror(0))
design.add_feature('comb_long2', comb_long_2.set_layer('layer_shallow').set_mirror(0))
design.add_feature('comb_teeth_1', comb_teeth_1.set_layer('layer_shallow').set_mirror(0))
design.add_feature('comb_teeth_2', comb_teeth_2.set_layer('layer_shallow').set_mirror(0))

align1, align1b = Feature.align_mark_squares(pos = [leftlim+1000,y_main_OF-1000])
design.add_feature('align_mark1', align1.set_layer('layer_shallow'))
design.add_feature('align_mark1b', align1b.set_layer('layer_growth'))

align2, align2b = Feature.align_mark_squares(pos = [leftlim+5000,y_main_OF-1000],rotation=True)
design.add_feature('align_mark2', align2.set_layer('layer_shallow'))
design.add_feature('align_mark2b', align2b.set_layer('layer_deep'))

#design.add_feature('numbers1', num_pad.set_layer('layer_growth'))
#design.add_feature('numbers2', num_pad2.set_layer('layer_growth'))
#design.add_feature('numbers3', num_pad3.set_layer('layer_growth'))
#design.add_feature('numbers4', num_pad4.set_layer('layer_growth'))


In [9]:
fulldesign = design.multiplicate(origins)

In [10]:
fulldesign.draw_design()

In [11]:
fulldesign.close()

In [12]:
#open drawing
drawing = dxf.drawing(file_name)
#define all layers
for x in layers:
    drawing.add_layer(layers[x]['name'], color=layers[x]['color'])

#add design at all positions defined by origins 
#!!!!!!!!!!!!VERY IMPORTANT!!!!!!!!!!
#(I use deepcopy here to be able to copy the same design several times)
#!!!!!!!!!!!!VERY IMPORTANT!!!!!!!!!!
for x in origins:
    design_adjusted = mf.set_design_origin(deepcopy(design),x)
    mf.draw_design(design_adjusted,drawing)

#complete design with global alignement marks
align, align2 = mf.align_mark_squares(pos = [-40000,1000])
drawing = mf.add_closed_polyline(layers['layer_shallow'],align,drawing)
drawing = mf.add_closed_polyline(layers['layer_growth'],align2,drawing)

align, align2 = mf.align_mark_squares(pos = [-40000,5000],rotation=True)
drawing = mf.add_closed_polyline(layers['layer_shallow'],align,drawing)
drawing = mf.add_closed_polyline(layers['layer_deep'],align2,drawing)

align, align2 = mf.align_mark_squares(pos = [40000,-3000])
drawing = mf.add_closed_polyline(layers['layer_shallow'],align,drawing)
drawing = mf.add_closed_polyline(layers['layer_growth'],align2,drawing)

align, align2 = mf.align_mark_squares(pos = [40000,-5000],rotation=True)
drawing = mf.add_closed_polyline(layers['layer_shallow'],align,drawing)
drawing = mf.add_closed_polyline(layers['layer_deep'],align2,drawing)

drawing.save()

If you want to directly draw some of the features in this notebook, just to have a look at them, you can use the following code

In [45]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()
from ipywidgets import SelectMultiple
from ipywidgets import interactive
from ipywidgets import interact
from ipywidgets import Checkbox
from bokeh.models import Range1d

Loading BokehJS ...

In [46]:
sel_mult = SelectMultiple(options = [x for x in design.layers])

In [47]:
def find_min(cur_min, new_min):
    if new_min<cur_min:
        cur_min = new_min
    return cur_min

def find_max(cur_max, new_max):
    if new_max>cur_max:
        cur_max = new_max
    return cur_max

def update_bounds(cur_bound, feature_coord):
    cur_bound[0] = find_min(cur_bound[0],np.min([np.min(x[:,0]) for x in feature_coord]))
    cur_bound[1] = find_max(cur_bound[1],np.max([np.max(x[:,0]) for x in feature_coord]))

    cur_bound[2] = find_min(cur_bound[2],np.min([np.min(x[:,1]) for x in feature_coord]))
    cur_bound[3] = find_max(cur_bound[3],np.max([np.max(x[:,1]) for x in feature_coord]))

    return cur_bound

def g(m):

    layers_to_plot = m
    if len(layers_to_plot)>0:
        features_to_plot = [x for x in design.features if design.features[x].layer in layers_to_plot]
        if len(features_to_plot)>0:
            #chk = [Checkbox(description=a) for a in features_to_plot]
            #interact(updatePlot, **{c.description: c.value for c in chk})
            sel_mult2 = SelectMultiple(options = features_to_plot)
            interactive_plot2 = interactive(updatePlot, n=sel_mult2)
            display(interactive_plot2)

def updatePlot(n):
    #clear_output()
    features_to_plot = n
    if len(features_to_plot)>0:
        p = figure(title="simple line example", x_axis_label='x', y_axis_label='y',output_backend="webgl")
        cur_bound = [100000,-100000,100000,-100000]
        for elem in features_to_plot:
            toplot = design.features[elem].coord
            cur_bound = update_bounds(cur_bound, toplot)
            for tp in toplot:
                p.line(tp[:,0], tp[:,1])
            if not design.features[elem].mirror == None:
                flipped = design.features[elem].flip_feature(0).coord
                cur_bound = update_bounds(cur_bound, flipped)
                for tp in flipped:
                    p.line(tp[:,0], tp[:,1])
                    
        maxrange = np.max([cur_bound[1]-cur_bound[0],cur_bound[3]-cur_bound[2]])
        p.x_range=Range1d(cur_bound[0],cur_bound[0]+maxrange)
        p.y_range=Range1d(cur_bound[2],cur_bound[2]+maxrange)
                
        plot_height = 800
        plot_width = 800
        show(p)

In [48]:
interactive_plot = interactive(g, m=sel_mult)
display(interactive_plot)

A Jupyter Widget

In [13]:
%matplotlib inline
import mpld3
mpld3.enable_notebook()

In [16]:
features_to_plot = ['inlet_channel','over_flow','back_flow','channel_flow','serpentine','punching_entry',
                   'punching_BF','punching_OF','punching_CF','dial_to_serpentine','serpentine_to_CF']
for elem in features_to_plot:
    
    toplot = design.features[elem].coord
    for tp in toplot:
        plt.plot(tp[:,0], tp[:,1])
        if not design.features[elem].mirror == None:
            flipped = mf.flip_feature(tp,0)
            plt.plot(flipped[:,0], flipped[:,1])
plt.axis('equal')
plt.show()


AttributeError: module 'microfluidics_class' has no attribute 'flip_feature'

In [10]:
from bokeh.plotting import figure, output_notebook, show

In [12]:
output_notebook()

Loading BokehJS ...

In [13]:
p = figure(title="simple line example", x_axis_label='x', y_axis_label='y')

In [37]:
layers_to_plot = ['layer_deep']
features_to_plot = [x for x in design.features if design.features[x].layer in layers_to_plot]
#features_to_plot = ['inlet_channel','over_flow','back_flow','channel_flow','serpentine','punching_entry',
                   #'punching_BF','punching_OF','punching_CF','dial_to_serpentine','serpentine_to_CF']
for elem in features_to_plot:
    #print(elem)
    toplot = design.features[elem].coord
    for tp in toplot:
        p.line(tp[:,0], tp[:,1])
    if not design.features[elem].mirror == None:
        flipped = design.features[elem].flip_feature(0).coord
        for tp in flipped:
            p.line(tp[:,0], tp[:,1])
#plt.axis('equal')
show(p)